In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [24]:
ant_master

,Name,version,wmc,dit,noc,cbo,rfc,lcom,ca,ce,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,prop-1,4,2,4,0,2,5,1,0,2,...,1.000000,0,0.941176,0.750000,1,1,10.500000,3,1.5000,0
1,prop-1,4,12,4,1,19,44,66,2,18,...,0.000000,0,0.905172,0.350000,2,3,20.583333,8,2.0833,0
2,prop-1,4,10,4,0,13,48,45,4,13,...,1.000000,0,0.918919,0.500000,2,3,19.100000,4,1.2000,0
3,prop-1,4,2,3,0,5,7,1,0,5,...,1.000000,0,0.933333,0.750000,1,1,11.500000,4,2.0000,1
4,prop-1,4,8,1,0,4,38,0,3,1,...,0.500000,0,0.000000,0.285714,0,0,30.625000,2,0.8750,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18466,prop-1,185,17,1,0,24,67,96,9,15,...,0.150000,0,0.000000,0.250000,0,0,41.529412,6,2.6471,1
18467,prop-1,185,9,1,0,25,49,14,7,18,...,0.333333,0,0.000000,0.319444,0,0,29.888889,3,1.3333,0
18468,prop-1,185,6,3,0,10,29,0,0,10,...,0.800000,0,0.818182,0.400000,0,0,45.500000,6,2.1667,0
18469,prop-1,185,5,5,0,28,9,10,26,2,...,1.000000,0,0.962963,0.650000,1,1,4.400000,1,0.2000,0


In [25]:
train, test = train_test_split(ant_master, test_size=0.2)

In [26]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [27]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
16174,3,1,0,16,73,1,0,16,2,0.900000,286,0.600000,0,0.000000,0.666667,0,0,92.666667,8,2.6667
17310,6,1,0,14,23,0,1,13,5,0.300000,127,1.000000,1,0.000000,0.583333,0,0,19.833333,2,1.1667
7243,0,1,0,0,0,0,0,0,0,2.000000,2,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000
3477,8,1,0,7,8,28,0,7,8,2.000000,8,0.000000,0,0.000000,0.500000,0,0,0.000000,1,1.0000
10313,2,4,0,1,4,1,0,1,1,2.000000,9,0.000000,0,0.933333,0.750000,1,1,3.500000,1,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12719,2,3,0,11,17,1,0,11,2,1.000000,145,0.000000,0,0.947368,0.600000,0,0,71.000000,1,0.5000
4421,3,3,0,6,19,1,0,6,1,1.000000,74,0.666667,0,0.928571,0.750000,1,1,22.666667,6,2.0000
15514,16,1,0,8,59,42,2,6,11,0.848485,543,1.000000,0,0.000000,0.293333,0,0,32.250000,6,1.8125
2906,3,5,0,3,5,3,1,2,3,1.500000,15,1.000000,0,0.962963,0.666667,1,1,3.666667,1,0.3333


In [28]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
5041,2,3,0,5,4,1,5,0,2,2.000000,11,1.000000,0,1.000000,0.666667,0,0,4.000000,0,0.0000
1101,4,4,0,18,39,4,0,18,2,0.933333,194,0.200000,0,0.920000,0.444444,1,3,46.250000,9,2.5000
14995,4,1,0,14,4,6,12,2,4,2.000000,4,0.000000,0,0.000000,0.875000,0,0,0.000000,1,1.0000
14466,5,2,2,9,35,8,2,7,1,1.166667,243,1.000000,0,0.823529,0.750000,0,0,46.400000,5,1.4000
2901,5,5,0,56,9,10,54,2,5,1.250000,28,1.000000,0,0.962963,0.650000,1,1,4.400000,1,0.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,13,1,0,18,68,46,1,17,12,0.891667,492,0.800000,2,0.000000,0.277778,0,0,36.076923,3,1.2308
13061,1,1,0,0,5,0,0,0,0,2.000000,20,0.000000,0,0.000000,0.000000,0,0,12.000000,0,0.0000
373,2,1,0,6,10,0,1,5,2,0.000000,28,1.000000,0,0.000000,0.666667,0,0,12.500000,1,0.5000
2040,39,1,0,23,39,741,20,3,39,2.000000,39,0.000000,0,0.000000,0.623932,0,0,0.000000,1,1.0000


In [29]:
y_train = train['bug']
y_test = test['bug']

In [30]:
y_train

16174    0
17310    0
7243     0
3477     0
10313    0
        ..
12719    0
4421     0
15514    1
2906     0
8427     0
Name: bug, Length: 14776, dtype: int64

In [31]:
y_test

5041     0
1101     0
14995    0
14466    0
2901     0
        ..
7222     0
13061    0
373      0
2040     2
3167     0
Name: bug, Length: 3695, dtype: int64

In [32]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [33]:
x_train_np

array([[ 3.        ,  1.        ,  0.        , ..., 92.66666667,
         8.        ,  2.6667    ],
       [ 6.        ,  1.        ,  0.        , ..., 19.83333333,
         2.        ,  1.1667    ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [16.        ,  1.        ,  0.        , ..., 32.25      ,
         6.        ,  1.8125    ],
       [ 3.        ,  5.        ,  0.        , ...,  3.66666667,
         1.        ,  0.3333    ],
       [ 5.        ,  2.        ,  0.        , ..., 24.8       ,
         5.        ,  2.        ]])

In [34]:
x_test_np

array([[ 2.        ,  3.        ,  0.        , ...,  4.        ,
         0.        ,  0.        ],
       [ 4.        ,  4.        ,  0.        , ..., 46.25      ,
         9.        ,  2.5       ],
       [ 4.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 2.        ,  1.        ,  0.        , ..., 12.5       ,
         1.        ,  0.5       ],
       [39.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [31.        ,  3.        ,  0.        , ..., 23.48387097,
        23.        ,  2.4839    ]])

In [35]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [36]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [37]:
train_x

array([[[ 3.        ,  1.        ,  0.        , ..., 92.66666667,
          8.        ,  2.6667    ]],

       [[ 6.        ,  1.        ,  0.        , ..., 19.83333333,
          2.        ,  1.1667    ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       ...,

       [[16.        ,  1.        ,  0.        , ..., 32.25      ,
          6.        ,  1.8125    ]],

       [[ 3.        ,  5.        ,  0.        , ...,  3.66666667,
          1.        ,  0.3333    ]],

       [[ 5.        ,  2.        ,  0.        , ..., 24.8       ,
          5.        ,  2.        ]]])

In [38]:
test_y

array([0, 0, 0, ..., 0, 2, 0])

In [39]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.00001), metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [40]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 200, callbacks=[tensorboard_callback])

Epoch 1/200
462/462 [==============================] - 7s 5ms/step - loss: 1.3606 - mse: 1.3606 - mae: 0.3160 - root_mean_squared_error: 1.1665 - mean_squared_logarithmic_error: 0.1646
Epoch 2/200
462/462 [==============================] - 2s 5ms/step - loss: 1.3265 - mse: 1.3265 - mae: 0.3588 - root_mean_squared_error: 1.1518 - mean_squared_logarithmic_error: 0.1526
Epoch 3/200
462/462 [==============================] - 2s 5ms/step - loss: 1.2903 - mse: 1.2903 - mae: 0.4199 - root_mean_squared_error: 1.1359 - mean_squared_logarithmic_error: 0.1483
Epoch 4/200
462/462 [==============================] - 2s 5ms/step - loss: 1.2741 - mse: 1.2741 - mae: 0.4631 - root_mean_squared_error: 1.1288 - mean_squared_logarithmic_error: 0.1528
Epoch 5/200
462/462 [==============================] - 2s 5ms/step - loss: 1.2674 - mse: 1.2674 - mae: 0.4810 - root_mean_squared_error: 1.1258 - mean_squared_logarithmic_error: 0.1561
Epoch 6/200
462/462 [==============================] - 2s 5ms/step - loss: 

462/462 [==============================] - 2s 5ms/step - loss: 1.0544 - mse: 1.0544 - mae: 0.3996 - root_mean_squared_error: 1.0268 - mean_squared_logarithmic_error: 0.1315
Epoch 45/200
462/462 [==============================] - 2s 5ms/step - loss: 1.0512 - mse: 1.0512 - mae: 0.4022 - root_mean_squared_error: 1.0253 - mean_squared_logarithmic_error: 0.1321
Epoch 46/200
462/462 [==============================] - 2s 5ms/step - loss: 1.0497 - mse: 1.0497 - mae: 0.4017 - root_mean_squared_error: 1.0245 - mean_squared_logarithmic_error: 0.1320
Epoch 47/200
462/462 [==============================] - 2s 5ms/step - loss: 1.0476 - mse: 1.0476 - mae: 0.3980 - root_mean_squared_error: 1.0235 - mean_squared_logarithmic_error: 0.1309
Epoch 48/200
462/462 [==============================] - 2s 5ms/step - loss: 1.0461 - mse: 1.0461 - mae: 0.3987 - root_mean_squared_error: 1.0228 - mean_squared_logarithmic_error: 0.1304
Epoch 49/200
462/462 [==============================] - 2s 5ms/step - loss: 1.0407 

462/462 [==============================] - 2s 5ms/step - loss: 0.9943 - mse: 0.9943 - mae: 0.3948 - root_mean_squared_error: 0.9971 - mean_squared_logarithmic_error: 0.1267
Epoch 89/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9913 - mse: 0.9913 - mae: 0.3920 - root_mean_squared_error: 0.9957 - mean_squared_logarithmic_error: 0.1258
Epoch 90/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9900 - mse: 0.9900 - mae: 0.3949 - root_mean_squared_error: 0.9950 - mean_squared_logarithmic_error: 0.1265
Epoch 91/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9921 - mse: 0.9921 - mae: 0.3897 - root_mean_squared_error: 0.9960 - mean_squared_logarithmic_error: 0.1249
Epoch 92/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9900 - mse: 0.9900 - mae: 0.3969 - root_mean_squared_error: 0.9950 - mean_squared_logarithmic_error: 0.1266
Epoch 93/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9894 

462/462 [==============================] - 2s 5ms/step - loss: 0.9630 - mse: 0.9630 - mae: 0.3849 - root_mean_squared_error: 0.9813 - mean_squared_logarithmic_error: 0.1219
Epoch 132/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9614 - mse: 0.9614 - mae: 0.3905 - root_mean_squared_error: 0.9805 - mean_squared_logarithmic_error: 0.1231
Epoch 133/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9583 - mse: 0.9583 - mae: 0.3873 - root_mean_squared_error: 0.9789 - mean_squared_logarithmic_error: 0.1225
Epoch 134/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9640 - mse: 0.9640 - mae: 0.3877 - root_mean_squared_error: 0.9818 - mean_squared_logarithmic_error: 0.1227
Epoch 135/200
462/462 [==============================] - 2s 5ms/step - loss: 0.9628 - mse: 0.9628 - mae: 0.3865 - root_mean_squared_error: 0.9812 - mean_squared_logarithmic_error: 0.1220
Epoch 136/200
462/462 [==============================] - 2s 5ms/step - loss: 0.

462/462 [==============================] - 2s 4ms/step - loss: 0.9453 - mse: 0.9453 - mae: 0.3828 - root_mean_squared_error: 0.9722 - mean_squared_logarithmic_error: 0.1208
Epoch 175/200
462/462 [==============================] - 2s 4ms/step - loss: 0.9416 - mse: 0.9416 - mae: 0.3852 - root_mean_squared_error: 0.9704 - mean_squared_logarithmic_error: 0.1207
Epoch 176/200
462/462 [==============================] - 2s 4ms/step - loss: 0.9417 - mse: 0.9417 - mae: 0.3856 - root_mean_squared_error: 0.9704 - mean_squared_logarithmic_error: 0.1210
Epoch 177/200
462/462 [==============================] - 2s 4ms/step - loss: 0.9397 - mse: 0.9397 - mae: 0.3826 - root_mean_squared_error: 0.9694 - mean_squared_logarithmic_error: 0.1202
Epoch 178/200
462/462 [==============================] - 2s 4ms/step - loss: 0.9381 - mse: 0.9381 - mae: 0.3834 - root_mean_squared_error: 0.9685 - mean_squared_logarithmic_error: 0.1202
Epoch 179/200
462/462 [==============================] - 2s 4ms/step - loss: 0.

In [41]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:48:36 ago. (Use '!kill 3750' to kill it.)

In [42]:
score = model.evaluate(test_x, test_y)
score

116/116 [==============================] - 1s 2ms/step - loss: 1.0397 - mse: 1.0397 - mae: 0.3770 - root_mean_squared_error: 1.0196 - mean_squared_logarithmic_error: 0.1277


[1.0396785736083984,
 1.0396785736083984,
 0.37704965472221375,
 1.0196462869644165,
 0.12771815061569214]